<a href="https://colab.research.google.com/github/jnrahul92/NLP_IN_ACTION_BOOK/blob/main/NLP_in_Action_Chapter_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
topic = {}

In [4]:
import numpy as np
tfidf = dict(list(zip('cat dog apple lion NYC love'.split(),np.random.rand(6))))

In [5]:
topic['petness'] = (0.3*tfidf['cat']+0.3*tfidf['dog']+0*tfidf['lion']+0*tfidf['apple']-0.2*tfidf['NYC']+0.2*tfidf['love'])

In [6]:
topic

{'petness': 0.10539045087180754}

In [7]:
tfidf

{'cat': 0.24791995654963261,
 'dog': 0.42145336470568107,
 'apple': 0.8751355065441775,
 'lion': 0.7517660360822533,
 'NYC': 0.7384284783452286,
 'love': 0.2613207508212958}

In [8]:
import pandas as pd
pd.options.display.width = 120

In [9]:
sms = pd.read_csv("sms-spam.csv")

In [10]:
index = ['sms{}{}'.format(i,'!'*j) for (i,j) in zip(range(len(sms)),sms.spam)]

In [11]:
sms = pd.DataFrame(sms.values,columns = sms.columns,index=index)

In [12]:
sms['spam'] = sms.spam.astype(int)

In [13]:
len(sms)

4837

In [14]:
sms.spam.sum()

638

In [15]:
sms.head(6)

,Unnamed: 0,spam,text
sms0,0,0,"Go until jurong point, crazy.. Available only ..."
sms1,1,0,Ok lar... Joking wif u oni...
sms2!,2,1,Free entry in 2 a wkly comp to win FA Cup fina...
sms3,3,0,U dun say so early hor... U c already then say...
sms4,4,0,"Nah I don't think he goes to usf, he lives aro..."
sms5!,5,1,FreeMsg Hey there darling it's been 3 week's n...


In [16]:
sms.drop(columns=['Unnamed: 0'],inplace = True
         )

In [17]:
sms.columns

Index(['spam', 'text'], dtype='object')

In [18]:
sms.head(5)

,spam,text
sms0,0,"Go until jurong point, crazy.. Available only ..."
sms1,0,Ok lar... Joking wif u oni...
sms2!,1,Free entry in 2 a wkly comp to win FA Cup fina...
sms3,0,U dun say so early hor... U c already then say...
sms4,0,"Nah I don't think he goes to usf, he lives aro..."


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize
tfidf_model = TfidfVectorizer(tokenizer = casual_tokenize)

In [20]:
tfidf_docs = tfidf_model.fit_transform(raw_documents=sms.text).toarray()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [21]:
tfidf_docs.shape

(4837, 9232)

In [22]:
sms.spam.sum()

638

In [23]:
mask = sms.spam.astype(bool).values

In [24]:
spam_centroid = tfidf_docs[mask].mean(axis = 0)
ham_centroid = tfidf_docs[~mask].mean(axis = 0)

In [25]:
spam_centroid.round(2)

array([0.06, 0.  , 0.  , ..., 0.  , 0.  , 0.  ])

In [26]:
ham_centroid.round(2)

array([0.02, 0.01, 0.  , ..., 0.  , 0.  , 0.  ])

In [27]:
spamminess_score = tfidf_docs.dot(spam_centroid - ham_centroid)

In [28]:
spamminess_score.round(2)

array([-0.01, -0.02,  0.04, ..., -0.01, -0.  ,  0.  ])

In [29]:
from sklearn.preprocessing import MinMaxScaler

In [30]:
sms['lda_score'] = MinMaxScaler().fit_transform(spamminess_score.reshape(-1,1))

In [31]:
sms['lda_predict'] = (sms.lda_score > 0.5).astype(int)

In [32]:
sms.head(6)

,spam,text,lda_score,lda_predict
sms0,0,"Go until jurong point, crazy.. Available only ...",0.227478,0
sms1,0,Ok lar... Joking wif u oni...,0.177888,0
sms2!,1,Free entry in 2 a wkly comp to win FA Cup fina...,0.718785,1
sms3,0,U dun say so early hor... U c already then say...,0.184565,0
sms4,0,"Nah I don't think he goes to usf, he lives aro...",0.286944,0
sms5!,1,FreeMsg Hey there darling it's been 3 week's n...,0.548003,1


In [33]:
len(tfidf_model.vocabulary_)

9232

In [34]:
tfidf_docs = pd.DataFrame(tfidf_docs)

In [35]:
tfidf_docs = tfidf_docs - tfidf_docs.mean()

In [36]:
tfidf_docs.shape

(4837, 9232)

In [37]:
from sklearn.decomposition import PCA

In [38]:
pca = PCA(n_components=16)

In [39]:
pca = pca.fit(tfidf_docs)

In [40]:
pca_topic_vectors = pca.transform(tfidf_docs)

In [41]:
colums = ['topic{}'.format(i) for i in range(pca.n_components_)]

In [42]:
pca_topic_vectors = pd.DataFrame(pca_topic_vectors, columns = colums, index = index)

In [43]:
pca_topic_vectors.round(3).head(6)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15
sms0,0.201,0.003,0.037,0.011,-0.019,-0.053,0.039,-0.065,0.012,-0.083,0.006,-0.006,0.003,-0.038,-0.010,0.038
sms1,0.404,-0.094,-0.078,0.051,0.100,0.047,0.023,0.065,0.024,-0.025,-0.005,0.034,0.043,-0.020,0.054,-0.027
sms2!,-0.030,-0.048,0.090,-0.067,0.091,-0.043,-0.000,-0.002,-0.057,0.051,0.126,0.026,0.024,-0.027,-0.051,0.059
sms3,0.329,-0.033,-0.035,-0.016,0.052,0.056,-0.166,-0.074,0.064,-0.109,0.016,0.022,0.072,-0.035,0.026,-0.072
sms4,0.002,0.031,0.038,0.034,-0.075,-0.092,-0.044,0.062,-0.044,0.028,0.030,-0.014,0.030,0.039,-0.076,-0.008
sms5!,-0.016,0.059,0.014,-0.006,0.122,-0.040,0.005,0.168,-0.022,0.063,0.041,0.054,-0.039,0.076,-0.006,-0.005


In [44]:
column_nums, terms = zip(*sorted(zip(tfidf_model.vocabulary_.values(),tfidf_model.vocabulary_.keys())))

In [45]:
weights = pd.DataFrame(pca.components_,columns = terms,index=['topic{}'.format(i) for i in range(16)])

In [46]:
pd.options.display.max_columns = 8

In [47]:
weights.head(4).round(3)

,!,"""",#,#150,...,…,┾,〨ud,鈥
topic0,-0.071,0.008,-0.001,-0.000,...,-0.002,0.001,0.001,0.001
topic1,0.064,0.008,0.000,-0.000,...,0.003,0.001,0.001,0.001
topic2,0.071,0.027,0.000,0.001,...,0.002,-0.001,-0.001,-0.001
topic3,-0.059,-0.032,-0.001,-0.000,...,0.001,0.001,0.001,0.001


In [48]:
pd.options.display.max_columns = 12

In [49]:
deals = weights['! ;) :) half off free crazy deal only $ 80 %'.split()].round(3)*100

In [50]:
deals

,!,;),:),half,off,free,crazy,deal,only,$,80,%
topic0,-7.1,0.1,-0.5,-0.0,-0.4,-2.0,-0.0,-0.1,-2.2,0.3,-0.0,-0.0
topic1,6.4,0.0,7.4,0.1,0.4,-2.3,-0.2,-0.1,-3.8,-0.1,-0.0,-0.2
topic2,7.1,0.2,-0.1,0.1,0.3,4.4,0.1,-0.1,0.7,0.0,0.0,0.1
topic3,-5.9,-0.3,-7.1,0.2,0.3,-0.2,0.0,0.1,-2.3,0.1,-0.1,-0.3
topic4,38.1,-0.1,-12.4,-0.1,-0.2,9.9,0.1,-0.2,3.0,0.3,0.1,-0.1
topic5,-26.5,0.1,-1.6,-0.3,-0.7,-1.4,-0.6,-0.2,-1.8,-0.9,0.0,0.0
topic6,-10.9,-0.5,19.9,-0.4,-0.9,-0.6,-0.2,-0.1,-1.4,-0.0,-0.0,-0.1
topic7,16.1,0.1,-18.1,0.8,0.8,-2.9,0.0,0.1,-1.9,-0.3,0.0,-0.1
topic8,34.4,0.1,4.8,-0.4,-0.5,-0.1,-0.4,-0.4,3.2,-0.6,-0.0,-0.2
topic9,7.7,-0.3,16.7,1.5,-0.9,6.5,-0.5,-0.4,3.1,-0.5,-0.0,0.0


In [51]:
from sklearn.decomposition import TruncatedSVD

In [52]:
svd = TruncatedSVD(n_components=16, n_iter = 100)

In [53]:
svd_topic_vectors = svd.fit_transform(tfidf_docs.values)

In [54]:
svd_topic_vectors = pd.DataFrame(svd_topic_vectors, columns = colums, index = index)

In [55]:
svd_topic_vectors.round(3).head(6)

,topic0,topic1,topic2,topic3,topic4,topic5,...,topic10,topic11,topic12,topic13,topic14,topic15
sms0,0.201,0.003,0.037,0.011,-0.019,-0.053,...,0.007,-0.007,0.002,-0.036,-0.014,0.037
sms1,0.404,-0.094,-0.078,0.051,0.100,0.047,...,-0.004,0.036,0.043,-0.021,0.051,-0.042
sms2!,-0.030,-0.048,0.090,-0.067,0.091,-0.043,...,0.125,0.023,0.026,-0.020,-0.042,0.052
sms3,0.329,-0.033,-0.035,-0.016,0.052,0.056,...,0.022,0.023,0.073,-0.046,0.022,-0.070
sms4,0.002,0.031,0.038,0.034,-0.075,-0.093,...,0.028,-0.009,0.027,0.034,-0.083,-0.021
sms5!,-0.016,0.059,0.014,-0.006,0.122,-0.040,...,0.041,0.055,-0.037,0.075,-0.001,0.020


In [56]:
svd_topic_vectors = (svd_topic_vectors.T/ np.linalg.norm(svd_topic_vectors, axis=1)).T

In [57]:
svd_topic_vectors.iloc[:10].dot(svd_topic_vectors.iloc[:10].T).round(1)

,sms0,sms1,sms2!,sms3,sms4,sms5!,sms6,sms7,sms8!,sms9!
sms0,1.0,0.6,-0.1,0.6,-0.0,-0.3,-0.3,-0.1,-0.3,-0.3
sms1,0.6,1.0,-0.2,0.8,-0.2,0.0,-0.2,-0.2,-0.1,-0.1
sms2!,-0.1,-0.2,1.0,-0.2,0.1,0.4,0.0,0.3,0.5,0.4
sms3,0.6,0.8,-0.2,1.0,-0.2,-0.3,-0.1,-0.3,-0.2,-0.1
sms4,-0.0,-0.2,0.1,-0.2,1.0,0.2,0.0,0.1,-0.4,-0.2
sms5!,-0.3,0.0,0.4,-0.3,0.2,1.0,-0.1,0.1,0.3,0.4
sms6,-0.3,-0.2,0.0,-0.1,0.0,-0.1,1.0,0.1,-0.2,-0.2
sms7,-0.1,-0.2,0.3,-0.3,0.1,0.1,0.1,1.0,0.1,0.4
sms8!,-0.3,-0.1,0.5,-0.2,-0.4,0.3,-0.2,0.1,1.0,0.3
sms9!,-0.3,-0.1,0.4,-0.1,-0.2,0.4,-0.2,0.4,0.3,1.0


In [58]:
from sklearn.feature_extraction.text import CountVectorizer

In [59]:
np.random.seed(42)

In [60]:
counter = CountVectorizer(tokenizer = casual_tokenize)

In [61]:
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=sms.text).toarray(),index = index)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [63]:
column_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(),counter.vocabulary_.keys())))

In [64]:
bow_docs.columns = terms

In [65]:
bow_docs.head()

,!,"""",#,#150,#5000,$,...,’,“,…,┾,〨ud,鈥
sms0,0,0,0,0,0,0,...,0,0,0,0,0,0
sms1,0,0,0,0,0,0,...,0,0,0,0,0,0
sms2!,0,0,0,0,0,0,...,0,0,0,0,0,0
sms3,0,0,0,0,0,0,...,0,0,0,0,0,0
sms4,0,0,0,0,0,0,...,0,0,0,0,0,0


In [66]:
from sklearn.decomposition import LatentDirichletAllocation as LDiA

In [67]:
ldia = LDiA(n_components=16, learning_method='batch')

In [68]:
ldia = ldia.fit(bow_docs)

In [69]:
ldia.components_.shape

(16, 9232)

In [70]:
pd.set_option("display.width",75)

In [71]:
components = pd.DataFrame(ldia.components_.T, index = terms, columns=colums)

In [72]:
components.round(2).head()

,topic0,topic1,topic2,topic3,topic4,topic5,...,topic10,topic11,topic12,topic13,topic14,topic15
!,184.03,15.00,72.22,394.95,45.48,36.14,...,37.42,44.18,64.40,297.29,41.16,11.70
"""",0.68,4.22,2.41,0.06,152.35,0.06,...,8.42,11.42,0.07,62.72,12.27,0.06
#,0.06,0.06,0.06,0.06,0.06,2.07,...,0.06,0.06,1.07,4.05,0.06,0.06
#150,0.06,0.06,0.06,0.06,0.06,0.06,...,0.06,0.06,0.06,0.06,0.06,0.06
#5000,0.06,0.06,0.06,0.06,0.06,0.06,...,0.06,0.06,0.06,0.06,0.06,0.06


In [73]:
components.topic3.sort_values(ascending=False)[:10]

!       394.952246
.       218.049724
to      119.533134
u       118.857546
call    111.948541
£       107.358914
,        96.954384
*        90.314783
your     90.215961
is       75.750037
Name: topic3, dtype: float64

In [74]:
ldia16_topic_vectors = ldia.transform(bow_docs)

In [75]:
ldia16_topic_vectors = pd.DataFrame(ldia16_topic_vectors, index = index, columns = colums)

In [76]:
ldia16_topic_vectors.round(2).head()

,topic0,topic1,topic2,topic3,topic4,topic5,...,topic10,topic11,topic12,topic13,topic14,topic15
sms0,0.00,0.62,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00
sms1,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.12,0.01,0.01,0.01,0.01
sms2!,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00
sms3,0.00,0.00,0.00,0.00,0.09,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00
sms4,0.39,0.00,0.33,0.00,0.00,0.00,...,0.00,0.00,0.09,0.00,0.00,0.00


In [77]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [78]:
from sklearn.model_selection import train_test_split

In [79]:
X_train, X_test, y_train, y_test = train_test_split(ldia16_topic_vectors, sms.spam,test_size = 0.5, random_state = 42)

In [80]:
lda = LDA(n_components=1)

In [81]:
lda = lda.fit(X_train, y_train)

In [82]:
sms['ldia16_spam'] = lda.predict(ldia16_topic_vectors)

In [83]:
round(float(lda.score(X_test, y_test)),2)

0.92

In [84]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_docs, sms.spam.values, test_size = 0.5, random_state = 42)

In [85]:
lda = LDA(n_components=1)

In [86]:
lda = lda.fit(X_train, y_train)

In [87]:
round(float(lda.score(X_train, y_train)),2)

1.0

In [88]:
round(float(lda.score(X_test, y_test)),2)

0.73

In [89]:
ldia32 = LDiA(n_components=32, learning_method='batch')

In [90]:
ldia32 = ldia32.fit(bow_docs)

In [91]:
ldia32_topic_vectors = ldia32.transform(bow_docs)

In [94]:
columns32 = ['topic{}'.format(i) for i in range(ldia32.n_components)]

In [95]:
ldia32_topic_vectors = pd.DataFrame(ldia32_topic_vectors, index = index, columns = columns32)

In [96]:
X_train, X_test, y_train, y_test = train_test_split(ldia32_topic_vectors, sms.spam,test_size = 0.5, random_state = 42)

In [97]:
lda = LDA(n_components=1)
lda = lda.fit(X_train, y_train)

In [98]:
sms['ldia32_spam'] = lda.predict(ldia32_topic_vectors)

In [99]:
round(float(lda.score(X_train, y_train)),2)

0.94

In [100]:
round(float(lda.score(X_test, y_test)),2)

0.93